# Complex queries
## An example use of the Digital Twin

In this notebook we are going to dive deep into queries:
* Examining the customer experience through the lens of different aspects of the experience. 

In our previous steps we made:
* Patrons, that have a `customer satisfaction`, a relationship with tickets, and locations.
* Tickets that are `owned by customers`
* Lines that lead to areas
* Areas where people are located. 

We will be doing a bunch of different queries on this theme. 


[This is the SDK repo on Github](https://github.com/Azure/azure-sdk-for-python/tree/4559e19e2f3146a49f1eba1706bb798071f4a1f5/sdk/digitaltwins/azure-digitaltwins-core)

[Here is the doc on the query language](https://docs.microsoft.com/en-us/azure/digital-twins/concepts-query-language)


In [1]:
from azure.identity import AzureCliCredential
from azure.digitaltwins.core import DigitalTwinsClient

# using yaml instead of 
import yaml
import uuid

# using altair instead of matplotlib for vizuals
import numpy as np
import pandas as pd

# you will get this from the ADT resource at portal.azure.com
your_digital_twin_url = "home-test-twin.api.wcus.digitaltwins.azure.net"

azure_cli = AzureCliCredential()
service_client = DigitalTwinsClient(
    your_digital_twin_url, azure_cli)
service_client

I'm going to set up a generic function that runs queries and gets the data. This will keep me from doing it over and over. 

**Note that with really large models this might perform poorly** I'm only doing this here as this example is very small. 

In [2]:
def query_ADT(query):
    query_result = service_client.query_twins(query)
    values = [i for i in query_result]
    return values

def query_to_df(query):
    query_result = query_ADT(query)
    values = pd.DataFrame(query_result)
    return values

query_expression = "SELECT * FROM digitaltwins"
query_to_df(query_expression)

,$dtId,$etag,satisfaction,totalWaitTime,$metadata,event_title,state,ticket_location,capacity,status
0,customer-cc04f3b6-39b0-4cef-bfff-a7d668cce446,"W/""158b01df-d555-4c1b-bf44-9c1e8c63eb02""",10.0,10.0,"{'$model': 'dtmi:billmanh:patron;1', 'satisfac...",NaN,NaN,NaN,NaN,NaN
1,customer-3cbd5e60-957d-44ff-944f-9adb42a20a52,"W/""5d7359d5-e0f6-44e7-80f8-2a3f191295f2""",10.0,10.0,"{'$model': 'dtmi:billmanh:patron;1', 'satisfac...",NaN,NaN,NaN,NaN,NaN
2,customer-5c454e2f-f70b-4352-b75a-958f1a49beba,"W/""d7a8af67-7b40-4ed8-94c4-553fb639e197""",7.0,0.0,"{'$model': 'dtmi:billmanh:patron;1', 'satisfac...",NaN,NaN,NaN,NaN,NaN
3,customer-26196fee-5ffd-457a-86b7-192a998f3cf2,"W/""ee1d4d68-aa14-4d35-a029-7635675f24d7""",9.0,0.0,"{'$model': 'dtmi:billmanh:patron;1', 'satisfac...",NaN,NaN,NaN,NaN,NaN
4,customer-e6f49d8a-711b-41c3-9db8-c7ece3dbc32c,"W/""d1242800-1c21-4bab-909f-8dc49ef0ce92""",7.0,0.0,"{'$model': 'dtmi:billmanh:patron;1', 'satisfac...",NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
117,ticket-dc48bc69-b1f5-4850-b5af-037c4ea72961,"W/""5ea29c9e-a6be-4c02-979a-df1b2aa8cb5c""",NaN,NaN,"{'$model': 'dtmi:billmanh:ticket;1', 'event_ti...",Smashing Pumpkins,open,4,NaN,NaN
118,ticket-92826a40-b024-4b91-a4d2-7e20dcfddce3,"W/""002990f3-4ed8-464c-a90e-4b83ae5362fb""",NaN,NaN,"{'$model': 'dtmi:billmanh:ticket;1', 'event_ti...",Foo Fighters,open,1,NaN,NaN
119,ticket-8c483b5c-37d4-4027-8d67-d920bf8ca063,"W/""22d419c0-a6da-4935-82e3-5b1f7236908e""",NaN,NaN,"{'$model': 'dtmi:billmanh:ticket;1', 'event_ti...",Foo Fighters,open,2,NaN,NaN
120,ticket-09028447-8e23-4e4f-a100-098a4dc8e919,"W/""39cc602c-a41e-47bf-8da9-1a401419832a""",NaN,NaN,"{'$model': 'dtmi:billmanh:ticket;1', 'event_ti...",Foo Fighters,open,3,NaN,NaN


Note that the larger query will give you back all of the values, so you can pop it into a dataframe and filter on the `$metadata` to get the values you want

## IS_OF_MODEL
The process for most analysis is to query the items that are relevant into a dataframe and do your analysis on them. 

In [3]:
query_expression = "SELECT * FROM digitaltwins where IS_OF_MODEL('dtmi:mymodels:patron;1')"
customers = query_to_df(query_expression)
customers.satisfaction.describe()

count    84.000000
mean      7.142857
std       1.529965
min       5.000000
25%       6.000000
50%       7.000000
75%       8.250000
max      10.000000
Name: satisfaction, dtype: float64

In [4]:
query_expression ="""
SELECT T, CT
FROM DIGITALTWINS T
JOIN CT RELATED T.locatedIn
WHERE CT.$dtId = 'line-2'
"""

customers_in_area_2 = query_to_df(query_expression)
customers_in_area_2

,T,CT
0,{'$dtId': 'customer-cc04f3b6-39b0-4cef-bfff-a7...,"{'$dtId': 'line-2', '$etag': 'W/""dd64cd24-6ec2..."
1,{'$dtId': 'customer-25e19268-3433-4f09-afe3-94...,"{'$dtId': 'line-2', '$etag': 'W/""dd64cd24-6ec2..."
2,{'$dtId': 'customer-c87adbfa-1c6e-4ea9-9f03-83...,"{'$dtId': 'line-2', '$etag': 'W/""dd64cd24-6ec2..."


Ok let's unpack that:

`SELECT T, CT` - means to give short names to all the different classes, so in this case `T` refers to all twins, and CT refers to a related item. The related item comes out in the second column

`RELATED T.locatedIn` - gits all of elements that have a `locatedIn` relationship and stores it in `CT`. 

`WHERE CT.$dtId = 'line-2'` - limits the query to items that have that relationship with `line-2`. This is the filter part. 

**Note** it seems that all _joined queries_ require a specific twin by name. 



In [5]:
customers_in_area_2.loc[0]

T     {'$dtId': 'customer-cc04f3b6-39b0-4cef-bfff-a7...
CT    {'$dtId': 'line-2', '$etag': 'W/"dd64cd24-6ec2...
Name: 0, dtype: object

In [6]:
customers_in_area_2.loc[0,'CT']

{'$dtId': 'line-2',
 '$etag': 'W/"dd64cd24-6ec2-452d-b17a-83ebe4bcd73a"',
 'capacity': 3,
 '$metadata': {'$model': 'dtmi:billmanh:line;1',
  'capacity': {'lastUpdateTime': '2020-11-30T00:10:31.5945931Z'}}}

So let's look at the customers in `line-2`

In [7]:
customers_in_area_2.loc[0,'T']

{'$dtId': 'customer-cc04f3b6-39b0-4cef-bfff-a7d668cce446',
 '$etag': 'W/"158b01df-d555-4c1b-bf44-9c1e8c63eb02"',
 'satisfaction': 10,
 'totalWaitTime': 10,
 '$metadata': {'$model': 'dtmi:billmanh:patron;1',
  'satisfaction': {'lastUpdateTime': '2020-11-19T02:17:30.1401999Z'},
  'totalWaitTime': {'lastUpdateTime': '2020-11-19T02:17:30.1401999Z'}}}

In [8]:
l2_cust = pd.DataFrame(customers_in_area_2['T'].tolist())
l2_cust

,$dtId,$etag,satisfaction,totalWaitTime,$metadata
0,customer-cc04f3b6-39b0-4cef-bfff-a7d668cce446,"W/""158b01df-d555-4c1b-bf44-9c1e8c63eb02""",10,10,"{'$model': 'dtmi:billmanh:patron;1', 'satisfac..."
1,customer-25e19268-3433-4f09-afe3-94f466313368,"W/""91509c0d-7dc9-465b-a735-2fde2f67fe02""",10,10,"{'$model': 'dtmi:billmanh:patron;1', 'satisfac..."
2,customer-c87adbfa-1c6e-4ea9-9f03-83e3877ef5fc,"W/""44f9fd9b-141f-4603-8a0b-ad8a3730ed0d""",8,0,"{'$model': 'dtmi:billmanh:patron;1', 'satisfac..."


In [9]:
l2_cust.satisfaction.describe()

count     3.000000
mean      9.333333
std       1.154701
min       8.000000
25%       9.000000
50%      10.000000
75%      10.000000
max      10.000000
Name: satisfaction, dtype: float64

In [10]:
customers.satisfaction.describe()

count    84.000000
mean      7.142857
std       1.529965
min       5.000000
25%       6.000000
50%       7.000000
75%       8.250000
max      10.000000
Name: satisfaction, dtype: float64

Customers in line 2 have higher satisfaction than customers in general. 

## How many people in each line

In [11]:
query = """
SELECT COUNT()  
FROM DIGITALTWINS T  
JOIN CT RELATED T.locatedIn  
WHERE CT.$dtId IN ['line-0','line-1','line-2', 'line-3']
"""

customers_in_lines = query_to_df(query)
customers_in_lines

,COUNT
0,5


In [12]:
query = """
SELECT COUNT()  
FROM DIGITALTWINS T  
JOIN CT RELATED T.locatedIn  
WHERE CT.$dtId IN ['line-2']
"""

customers_in_lines = query_to_df(query)
customers_in_lines

,COUNT
0,3


The rough part is that you can only get one count back, not a count per line, like you could with propper SQL. You also have to hard code all of your `$dtID` as they require literal values. Lame.

Here is the way around that. 

`SELECT line, customer` <- select the columns that you want the query to return

`AND IS_OF_MODEL(customer, 'dtmi:billmanh:patron;1')` <- specify that `customer` are twins of the `patron` model. 

You still have to hard type the names of the lines, or rooms or whatever, but it returns all of the customers in all of the lines. 

In [13]:
query = """
SELECT line, customer
FROM DIGITALTWINS customer
JOIN line RELATED customer.locatedIn
WHERE line.$dtId IN ['line-0','line-1','line-2', 'line-3']
AND IS_OF_MODEL(customer, 'dtmi:mymodels:patron;1')
"""

customers_in_lines = query_to_df(query)
customers_in_lines

,line,customer
0,"{'$dtId': 'line-1', '$etag': 'W/""aa93fee2-cba0...",{'$dtId': 'customer-e6f49d8a-711b-41c3-9db8-c7...
1,"{'$dtId': 'line-1', '$etag': 'W/""aa93fee2-cba0...",{'$dtId': 'customer-21e17d28-76c3-4c04-8df9-39...
2,"{'$dtId': 'line-2', '$etag': 'W/""dd64cd24-6ec2...",{'$dtId': 'customer-cc04f3b6-39b0-4cef-bfff-a7...
3,"{'$dtId': 'line-2', '$etag': 'W/""dd64cd24-6ec2...",{'$dtId': 'customer-25e19268-3433-4f09-afe3-94...
4,"{'$dtId': 'line-2', '$etag': 'W/""dd64cd24-6ec2...",{'$dtId': 'customer-c87adbfa-1c6e-4ea9-9f03-83...


Easy enough to munge it into a dataframe:

In [14]:
c_in_line = pd.concat(
    [pd.DataFrame(customers_in_lines['line'].tolist()),
    pd.DataFrame(customers_in_lines['customer'].tolist())],
    axis=1
)

cols = c_in_line.columns.tolist()
cols[0] = 'line'
cols[4] = 'customer'
c_in_line.columns = cols
c_in_line

,line,$etag,capacity,$metadata,customer,$etag,satisfaction,totalWaitTime,$metadata
0,line-1,"W/""aa93fee2-cba0-4bd4-a020-114f058b0f01""",3,"{'$model': 'dtmi:billmanh:line;1', 'capacity':...",customer-e6f49d8a-711b-41c3-9db8-c7ece3dbc32c,"W/""d1242800-1c21-4bab-909f-8dc49ef0ce92""",7,0,"{'$model': 'dtmi:billmanh:patron;1', 'satisfac..."
1,line-1,"W/""aa93fee2-cba0-4bd4-a020-114f058b0f01""",3,"{'$model': 'dtmi:billmanh:line;1', 'capacity':...",customer-21e17d28-76c3-4c04-8df9-396703692a68,"W/""c62357a9-f4e5-4ec6-9d60-08b0da1125a7""",8,0,"{'$model': 'dtmi:billmanh:patron;1', 'satisfac..."
2,line-2,"W/""dd64cd24-6ec2-452d-b17a-83ebe4bcd73a""",3,"{'$model': 'dtmi:billmanh:line;1', 'capacity':...",customer-cc04f3b6-39b0-4cef-bfff-a7d668cce446,"W/""158b01df-d555-4c1b-bf44-9c1e8c63eb02""",10,10,"{'$model': 'dtmi:billmanh:patron;1', 'satisfac..."
3,line-2,"W/""dd64cd24-6ec2-452d-b17a-83ebe4bcd73a""",3,"{'$model': 'dtmi:billmanh:line;1', 'capacity':...",customer-25e19268-3433-4f09-afe3-94f466313368,"W/""91509c0d-7dc9-465b-a735-2fde2f67fe02""",10,10,"{'$model': 'dtmi:billmanh:patron;1', 'satisfac..."
4,line-2,"W/""dd64cd24-6ec2-452d-b17a-83ebe4bcd73a""",3,"{'$model': 'dtmi:billmanh:line;1', 'capacity':...",customer-c87adbfa-1c6e-4ea9-9f03-83e3877ef5fc,"W/""44f9fd9b-141f-4603-8a0b-ad8a3730ed0d""",8,0,"{'$model': 'dtmi:billmanh:patron;1', 'satisfac..."


How many people are in each line:  

In [15]:
c_in_line.groupby('line').count()['customer']

line
line-1    2
line-2    3
Name: customer, dtype: int64

Which group of people has the highest satisfaction?

In [16]:
c_in_line.groupby('line').mean()['satisfaction']

line
line-1    7.500000
line-2    9.333333
Name: satisfaction, dtype: float64